# 在线学习的批处理操作

BrainTrace 框架的在线学习模块提供了两种高效的批处理策略，用于优化神经网络的训练过程：

- **手动批处理**：显式管理批量维度，模型状态形状为 `(B, M)`，其中 `B` 为批处理大小，`M` 为模型参数数量
- **自动批处理**：使用 `brainstate.transform.vmap` 函数实现向量化操作，模型状态保持单样本形状 `(M)`，通过自动向量化处理批量数据


**💡 批处理的内存和计算优势**

**1. 内存布局优化:**
```python
# 低效的内存布局（逐样本）
for i in range(128):
    process_sample(i)  # 128次内存分配

# 高效的内存布局（批处理）
process_batch(all_128_samples)  # 1次内存分配，连续存储
```

**2. 并行计算优势:**
```python
# CPU逐样本: 128 × 单样本时间
# GPU批处理: 约等于 单样本时间（理想情况下）
```

这种手动批处理方法的核心思想是：**显式控制批量维度，最大化并行计算效率**。虽然代码稍微复杂，但在大规模训练中能带来显著的性能提升。

本教程将详细对比这两种方法的实现差异、适用场景和性能特点。


In [1]:
import brainstate
import braintools
import braintrace
import brainunit as u
import jax

brainstate.environ.set(dt=1.0 * u.ms)  # 设置时间步长为1毫秒

## 准备工作：数据集+模型

首先，我们创建一个模拟的分类任务数据集。

In [2]:
# 数据集参数配置
n_time = 16  # Time steps
n_batch = 128      # 批处理大小
n_in = 100         # 输入特征维度
n_hidden = 200     # 隐藏层神经元数量
n_out = 10         # 输出类别数量

# 生成随机训练数据
xs = brainstate.random.rand(n_time, n_batch, n_in)  # Input data shape: (16, 128, 100)
ys = brainstate.random.randint(0, n_out, n_batch)   # 标签数据 shape: (128,)

其次，构建一个基于漏积分发放（Leaky Integrate-and-Fire）神经元的循环神经网，用于完成这个数据集的分类任务。

In [3]:
class LIFNet(brainstate.nn.Module):
    """
    LIF神经网络模型

    结构：输入层 -> LIF神经元层（带循环连接）-> 输出层
    """

    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()

        # LIF神经元层：模拟生物神经元的漏积分发放行为
        self.neu = brainpy.state.LIF(n_hidden)

        # 权重初始化策略
        rec_init = braintools.init.KaimingNormal(unit=u.mV)    # 循环连接权重
        ff_init = braintools.init.KaimingNormal(unit=u.mV)     # 前馈连接权重

        # 突触连接层：整合前馈输入和循环反馈
        self.syn = brainpy.state.DeltaProj(
            comm=braintrace.nn.Linear(
                n_in + n_hidden, n_hidden,
                # 连接权重矩阵：[前馈权重; 循环权重]
                w_init=u.math.concatenate([
                    ff_init([n_in, n_hidden]),
                    rec_init([n_hidden, n_hidden])
                ], axis=0),
                b_init=braintools.init.ZeroInit(unit=u.mV)
            ),
            post=self.neu
        )

        # 输出层：将脉冲活动转换为分类输出
        self.out = brainstate.nn.LeakyRateReadout(n_hidden, n_out)

    def update(self, x):
        """
        模型前向传播

        Args:
            x: 输入数据

        Returns:
            网络输出（分类logits）
        """
        # 整合当前输入和循环连接的脉冲反馈
        combined_input = u.math.concatenate([x, self.neu.get_spike()], axis=-1)
        self.syn(combined_input)

        # 返回当前时刻的输出
        return self.out(self.neu())

## 手动批处理

手动批处理需要如下几个要求：

1. 初始化模型为一个批次的模型状态，模型状态的形状为$\mathbb{R}^{B\times M}$，其中$B$是批处理大小，$M$是模型参数的数量。
2. 在每次调用模型的`.update`函数时，传入一个批次的样本数据，该数据的形状为$\mathbb{R}^{B\times D}$，其中$D$是样本数据的维度。
3. 初始化在线学习算法时，将``mode``参数设置为`brainstate.mixin.Batching()`，以启用手动批处理模式。或者使用``brainstate.environ.set(mode=brainstate.mixin.Batching())``来设置全局批处理模式。

### 核心特点

手动批处理模式要求开发者显式处理批量维度：

1. **状态初始化**：模型状态形状必须为 `(B, M)`
2. **数据格式**：输入数据形状为 `(B, D)`
3. **模式设置**：使用 `brainstate.mixin.Batching()` 启用批处理模式


### 具体示例

以下是一个简单的手动批处理的示例。

In [4]:
class TrainerManualBatching:
    """手动批处理训练器"""

    def __init__(self, n_in, n_hidden, n_out):
        self.model = LIFNet(n_in, n_hidden, n_out)
        self.optimizer = braintools.optim.Adam(lr=1e-3)
        # 注册可训练参数
        self.optimizer.register_trainable_weights(self.model.states(brainstate.ParamState))

    @brainstate.transform.jit(static_argnums=0)
    def train(self, inputs, targets):
        """
        单轮训练步骤

        Args:
            inputs: 输入序列 shape: (T, B, D)
            targets: 目标标签 shape: (B,)
        """
        # 步骤1：初始化批量模型状态
        brainstate.nn.init_all_states(self.model, batch_size=inputs.shape[1])

        # 步骤2：创建在线学习算法实例
        model = braintrace.ES_D_RTRL(
            self.model,
            decay_or_rank=0.9,                           # 资格轨迹衰减因子
            mode=brainstate.mixin.Batching()            # 启用手动批处理模式
        )

        # 步骤3：编译计算图（优化执行效率）
        model.compile_graph(inputs[0])

        # 步骤4：获取可训练参数
        weights = self.model.states(brainstate.ParamState)

        def _etrace_grad(inp):
            """计算单步的损失和梯度"""
            out = model(inp)
            loss = braintools.metric.softmax_cross_entropy_with_integer_labels(
                out, targets
            ).mean()
            return loss, out

        def _etrace_step(prev_grads, x):
            """资格轨迹梯度累积步骤"""
            # 计算当前步的梯度
            f_grad = brainstate.transform.grad(
                _etrace_grad, weights,
                has_aux=True, return_value=True
            )
            cur_grads, local_loss, out = f_grad(x)

            # 累积梯度（资格轨迹机制）
            next_grads = jax.tree.map(lambda a, b: a + b, prev_grads, cur_grads)
            return next_grads, (out, local_loss)

        # 步骤5：时序前向传播与梯度累积
        grads = jax.tree.map(u.math.zeros_like, weights.to_dict_values())
        grads, (outs, losses) = brainstate.transform.scan(_etrace_step, grads, inputs)

        # 步骤6：梯度裁剪与参数更新
        grads = brainstate.functional.clip_grad_norm(grads, 1.0)
        self.optimizer.update(grads)

        return losses.mean()

    def f_train(self, n_epochs, inputs, targets):
        """完整训练流程"""
        for epoch in range(n_epochs):
            loss = self.train(inputs, targets)
            print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}')

In [5]:
# 创建训练器并开始训练
trainer_manual = TrainerManualBatching(n_in, n_hidden, n_out)
trainer_manual.f_train(10, xs, ys)

Epoch 1/10, Loss: 4.2803
Epoch 2/10, Loss: 3.4519
Epoch 3/10, Loss: 3.0564
Epoch 4/10, Loss: 2.7560
Epoch 5/10, Loss: 2.7972
Epoch 6/10, Loss: 2.7649
Epoch 7/10, Loss: 2.6040
Epoch 8/10, Loss: 2.5192
Epoch 9/10, Loss: 2.4659
Epoch 10/10, Loss: 2.3979


### 代码详解

**数据流概览**

在深入分析之前，让我们先了解整个批处理的数据流：

```
输入数据: inputs(T, B, D) → 模型状态(B, M) → 批量计算 → 梯度累积 → 参数更新
```

其中：
- `T`: 时间步长数
- `B`: 批处理大小
- `D`: 输入特征维度
- `M`: 模型状态维度



**🔑 批处理操作的三个核心要素**

**1. 批量状态初始化 - 为什么这是关键？**

```python
brainstate.nn.init_all_states(self.model, batch_size=inputs.shape[1])
```

这一行代码实际上在做什么：

```python
# 原始状态（单样本）
神经元电压: V → shape (200,)
神经元脉冲: spike → shape (200,)

# 批处理后（128个样本）
神经元电压: V → shape (128, 200)
神经元脉冲: spike → shape (128, 200)
```

为什么必须这样做？

- RNN/SNN需要维护时间步之间的状态
- 批处理意味着同时处理128个独立的序列
- 每个序列都需要自己的状态副本

**2. Batching模式 - 算法如何感知批处理？**

```python
mode=brainstate.mixin.Batching()
```

这个参数告诉ES_D_RTRL算法：

```python
# 没有Batching模式的期望
输入: (100,) 单个样本
输出: (10,) 单个预测

# 有Batching模式的期望
输入: (128, 100) 批量样本
输出: (128, 10) 批量预测
```

**3. 时序批量处理 - 最复杂的部分**

```python
grads, (outs, losses) = brainstate.transform.scan(_etrace_step, grads, inputs)
```

这里发生的事情：

```python
# inputs shape: (50, 128, 100) - 50个时间步，每步128个样本

时间步 0: 处理 inputs[0] → (128, 100) → 更新128个状态 → 计算梯度
时间步 1: 处理 inputs[1] → (128, 100) → 更新128个状态 → 累积梯度
...
时间步 49: 处理 inputs[49] → (128, 100) → 更新128个状态 → 最终梯度
```

## 自动批处理

自动批处理主要使用`brainstate.transform.vmap`函数来实现。该函数可以将模型的更新函数向量化，从而实现批处理操作。

### 核心特点

自动批处理通过 `vmap` 函数实现向量化操作：

1. **状态管理**：模型状态保持单样本形状 `(M)`
2. **自动向量化**：`vmap` 自动处理批量维度映射
3. **代码简洁**：减少手动批处理的复杂性

### 具体示例

In [6]:
class TrainerAutoBatching:
    """自动批处理训练器"""

    def __init__(self, n_in, n_hidden, n_out):
        self.model = LIFNet(n_in, n_hidden, n_out)
        self.optimizer = braintools.optim.Adam(lr=1e-3)
        self.optimizer.register_trainable_weights(self.model.states(brainstate.ParamState))

    @brainstate.transform.jit(static_argnums=0)
    def train(self, inputs, targets):
        """
        单轮训练步骤（自动批处理版本）

        Args:
            inputs: 输入序列 shape: (T, B, D)
            targets: 目标标签 shape: (B,)
        """
        # 步骤1：创建在线学习算法实例（无需手动批处理模式）
        model = braintrace.ES_D_RTRL(self.model, decay_or_rank=0.9)

        # 步骤2：使用vmap创建批量状态初始化函数
        @brainstate.transform.vmap_new_states(
            axis_size=inputs.shape[1],                   # 批处理大小
            state_tag='new',                            # 状态标签（用于区分不同的状态组）
        )
        def init():
            """初始化单个样本的模型状态"""
            brainstate.nn.init_all_states(self.model)
            model.compile_graph(inputs[0, 0])           # 使用单个样本编译图

        # 执行批量初始化
        init()

        # 步骤3：创建向量化模型包装器
        vmap_model = brainstate.nn.Vmap(
            model,
            vmap_states='new'                           # 指定要向量化的状态组
        )

        # 步骤4：获取可训练参数
        weights = self.model.states(brainstate.ParamState)

        def _etrace_grad(inp):
            """计算单步的损失和梯度（自动向量化版本）"""
            out = vmap_model(inp)                       # 自动处理批量维度
            loss = braintools.metric.softmax_cross_entropy_with_integer_labels(
                out, targets
            ).mean()
            return loss, out

        def _etrace_step(prev_grads, x):
            """资格轨迹梯度累积步骤"""
            f_grad = brainstate.transform.grad(
                _etrace_grad, weights,
                has_aux=True, return_value=True
            )
            cur_grads, local_loss, out = f_grad(x)
            next_grads = jax.tree.map(lambda a, b: a + b, prev_grads, cur_grads)
            return next_grads, (out, local_loss)

        # 步骤5：时序前向传播与梯度累积
        grads = jax.tree.map(u.math.zeros_like, weights.to_dict_values())
        grads, (outs, losses) = brainstate.transform.scan(_etrace_step, grads, inputs)

        # 步骤6：梯度裁剪与参数更新
        grads = brainstate.functional.clip_grad_norm(grads, 1.0)
        self.optimizer.update(grads)

        return losses.mean()

    def f_train(self, n_epochs, inputs, targets):
        """完整训练流程"""
        for epoch in range(n_epochs):
            loss = self.train(inputs, targets)
            print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}')

In [7]:
# 创建训练器并开始训练
trainer_auto = TrainerAutoBatching(n_in, n_hidden, n_out)
trainer_auto.f_train(10, xs, ys)

Epoch 1/10, Loss: 4.5856
Epoch 2/10, Loss: 3.4918
Epoch 3/10, Loss: 3.1237
Epoch 4/10, Loss: 2.9487
Epoch 5/10, Loss: 2.7401
Epoch 6/10, Loss: 2.7286
Epoch 7/10, Loss: 2.6906
Epoch 8/10, Loss: 2.5279
Epoch 9/10, Loss: 2.5536
Epoch 10/10, Loss: 2.4651


### 代码详解


**核心思想对比**

在深入分析之前，让我们理解自动批处理与手动批处理的根本差异：

| 维度 | 手动批处理 | 自动批处理 |
|------|------------|------------|
| **状态管理** | 显式批量状态 `(B, M)` | 单样本状态 `(M)` + 自动向量化 |
| **计算方式** | 直接批量计算 | `vmap` 函数式映射 |
| **代码复杂度** | 需要处理批量维度 | 抽象掉批量细节 |

```python
# 概念对比
手动批处理思路: 创建128个神经元状态副本，同时计算
自动批处理思路: 定义单个神经元计算逻辑，自动复制128次
```


**关键步骤详细解析**


**步骤1：算法实例创建（简化模式）**

```python
# 步骤1：创建在线学习算法实例（无需手动批处理模式）
model = braintrace.ES_D_RTRL(self.model, decay_or_rank=0.9)
```

**自动批处理关键点：**

注意这里**没有**使用 `mode=brainstate.mixin.Batching()`：

```python
# 手动批处理版本
model = braintrace.ES_D_RTRL(
    self.model,
    decay_or_rank=0.9,
    mode=brainstate.mixin.Batching()  # 显式启用批处理
)

# 自动批处理版本
model = braintrace.ES_D_RTRL(self.model, decay_or_rank=0.9)
# 算法以为自己在处理单个样本！
```

**步骤2：vmap状态初始化（核心机制）- 状态向量化的魔法**

```python
# 步骤2：使用vmap创建批量状态初始化函数
@brainstate.transform.vmap_new_states(
    axis_size=inputs.shape[1],                   # 批处理大小
    state_tag='new',                            # 状态标签（用于区分不同的状态组）
)
def init():
    """初始化单个样本的模型状态"""
    brainstate.nn.init_all_states(self.model) # 只初始化1个！
    model.compile_graph(inputs[0, 0])           # 使用单个样本编译图

# 执行批量初始化
init()
```

这里的关键理解：
- 函数 `init()` 只知道如何初始化**1个样本**的状态
- `vmap_new_states` 自动将这个函数**复制128次**
- 结果：得到128个独立的状态副本，但代码只写了1个样本的逻辑
- 状态标签 `'new'` 便于区分不同的状态组，确保后面便利地抽取出这些批量初始化的状态


**步骤3：Vmap模型包装器 - 单样本→批量的自动转换**

```python
vmap_model = brainstate.nn.Vmap(model, vmap_states='new')

# 调用时的内部流程：
输入: (128, 100)
  ↓ vmap自动分解
128个并行计算: 每个处理 (100,) → (10,)
  ↓ vmap自动组合
输出: (128, 10)
```

当你调用 `vmap_model(inp)` 时：

```python
# inp.shape = (128, 100)

# Step 1: vmap分解输入
sample_0 = inp[0]   # (100,)
sample_1 = inp[1]   # (100,)
...
sample_127 = inp[127] # (100,)

# Step 2: 并行执行（概念上，实际可能向量化）
result_0 = model_with_state_0(sample_0)   # (10,)
result_1 = model_with_state_1(sample_1)   # (10,)
...
result_127 = model_with_state_127(sample_127) # (10,)

# Step 3: vmap组合输出
output = stack([result_0, result_1, ..., result_127])  # (128, 10)
```

**关键洞察：** 模型函数始终以为自己在处理单个样本，完全不知道批处理的存在！

这种设计的美妙之处在于：你可以用最简单直观的方式编写神经网络逻辑（单样本），然后自动获得高效的批处理能力。


## 总结

BrainTrace 的两种批处理策略各有优势。手动批处理提供了更精细的控制和更高的性能，适合生产环境的大规模训练；自动批处理则以简洁的API降低了实现复杂度，更适合研究和原型开发。

选择合适的批处理策略需要综合考虑具体的应用场景、性能要求和开发效率。建议在项目初期使用自动批处理快速验证想法，在性能优化阶段考虑迁移到手动批处理。